In [5]:
from scipy import stats
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder,OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import lazypredict
from lazypredict.Supervised import LazyRegressor
from sklearn.model_selection import GridSearchCV



Đọc và chia data

In [6]:
data = pd.read_csv("dataAfterCleaningBinhTan1.csv", delimiter=",")
data.head()

,chieuDai,chieuNgang,dienTich,Gia/m2,Phongngu,SoTang,PhongTam,Loai,GiayTo,TinhTrangNoiThat,Phuong
0,16.00,4.00,65.00,1570.42,2,2.00,2.00,"nhà ngõ, hẻm",đã có sổ,nội thất đầy đủ,phường bình hưng hoà b
1,15.00,4.00,60.00,3465.42,3,3.00,3.00,"nhà ngõ, hẻm",đã có sổ,nội thất cao cấp,phường bình trị đông a
2,10.00,4.00,72.00,2777.92,2,2.00,2.00,"nhà mặt phố, mặt tiền",đã có sổ,NaN,phường an lạc
3,18.00,4.00,72.00,3350.83,5,4.00,5.00,"nhà ngõ, hẻm",đã có sổ,nội thất cao cấp,phường bình trị đông a
4,8.12,8.12,66.00,3775.42,4,4.00,4.00,"nhà ngõ, hẻm",đã có sổ,NaN,phường bình hưng hòa


In [7]:
data.SoTang.unique()

array([ 2.,  3.,  4.,  1.,  6.,  5., 11.,  7., 10.])

In [8]:
data = data.drop_duplicates()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4924 entries, 0 to 4923
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   chieuDai          4924 non-null   float64
 1   chieuNgang        4924 non-null   float64
 2   dienTich          4924 non-null   float64
 3   Gia/m2            4924 non-null   float64
 4   Phongngu          4924 non-null   int64  
 5   SoTang            4924 non-null   float64
 6   PhongTam          4924 non-null   float64
 7   Loai              4923 non-null   object 
 8   GiayTo            4914 non-null   object 
 9   TinhTrangNoiThat  3237 non-null   object 
 10  Phuong            4924 non-null   object 
dtypes: float64(6), int64(1), object(4)
memory usage: 423.3+ KB


In [9]:
target = "Gia/m2"
x = data.drop(target, axis=1)
y = data[target]
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=1609)

In [10]:
print("train.shape: ",x_train.shape)
print("test.shape: ",x_test.shape)

train.shape:  (3939, 10)
test.shape:  (985, 10)


Xem unique của từng cột categorical

In [11]:
print("uniqueLoai:")
x_train.Loai.unique()

uniqueLoai:


array(['nhà mặt phố, mặt tiền', 'nhà ngõ, hẻm', 'nhà phố liền kề',
       'nhà biệt thự', nan], dtype=object)

In [12]:
print("uniqueGiayTo:")
x_train.GiayTo.unique()

uniqueGiayTo:


array(['đã có sổ', 'sổ chung / công chứng vi bằng', 'đang chờ sổ',
       'giấy tờ viết tay', nan, 'không có sổ'], dtype=object)

In [13]:
print("uniqueTinhTrangNoiThat:")
x_train.TinhTrangNoiThat.unique()

uniqueTinhTrangNoiThat:


array([nan, 'nội thất cao cấp', 'nội thất đầy đủ', 'hoàn thiện cơ bản',
       'bàn giao thô'], dtype=object)

In [14]:
print("uniquePhuong:")
x_train.Phuong.unique()

uniquePhuong:


array(['phường bình hưng hòa', 'phường bình trị đông',
       'phường bình hưng hoà a', 'phường bình hưng hoà b',
       'phường tân tạo', 'phường an lạc', 'phường bình trị đông a',
       'phường bình trị đông b', 'phường tân tạo a', 'phường an lạc a'],
      dtype=object)

In [15]:
## lấy ra list Phường theo thứ tự tăng dần của giá
df = pd.DataFrame(data)

def sorted_districts(col):
    table = df.groupby(col)['Gia/m2'].mean().reset_index()
  
    sorted_table = table.sort_values(by='Gia/m2')
    sorted_districts = sorted_table[col].tolist()
    return sorted_districts

list_sorted_Phuong = sorted_districts('Phuong')

In [16]:
for i in range(0, len(list_sorted_Phuong), 4):
    print(list_sorted_Phuong[i:i+4])

['phường tân tạo a', 'phường bình hưng hoà b', 'phường tân tạo', 'phường bình hưng hòa']
['phường bình trị đông a', 'phường bình trị đông', 'phường an lạc', 'phường bình hưng hoà a']
['phường an lạc a', 'phường bình trị đông b']


Tiền xử lý lần 2

In [17]:
Loai_values = ["unknown",'nhà ngõ, hẻm', 'nhà phố liền kề', 'nhà mặt phố, mặt tiền', 'nhà biệt thự']
GiayTo_values = ["unknown",'giấy tờ viết tay', 'không có sổ', 'đang chờ sổ', 
            'sổ chung / công chứng vi bằng', 'đã có sổ']
TinhTrangNoiThat_values = ["unknown",'bàn giao thô', 'hoàn thiện cơ bản', 'nội thất đầy đủ', 'nội thất cao cấp']


ord_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value="unknown")),
    ("encoder", OrdinalEncoder(categories=[Loai_values,GiayTo_values,TinhTrangNoiThat_values,list_sorted_Phuong]))
])

num_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])
preprocessor = ColumnTransformer(transformers=[
    ("num_features", num_transformer, ["chieuDai", "chieuNgang","dienTich","Phongngu","SoTang","PhongTam"]),
    ("ordinal_features", ord_transformer, ["Loai","GiayTo","TinhTrangNoiThat","Phuong"]),
])



In [18]:
x_train_preprocessor = preprocessor.fit_transform(x_train)
x_test_preprocessor = preprocessor.transform(x_test)
lazy_reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None )
models, predictions = lazy_reg.fit(x_train_preprocessor, x_test_preprocessor, y_train, y_test)
print(predictions)

  0%|          | 0/42 [00:00<?, ?it/s]

100%|██████████| 42/42 [00:08<00:00,  4.71it/s]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000666 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 554
[LightGBM] [Info] Number of data points in the train set: 3939, number of used features: 10
[LightGBM] [Info] Start training from score 3710.665454
                               Adjusted R-Squared  R-Squared     RMSE  \
Model                                                                   
RandomForestRegressor                        0.60       0.60   766.38   
XGBRegressor                                 0.59       0.59   775.41   
ExtraTreesRegressor                          0.58       0.58   783.87   
LGBMRegressor                                0.58       0.58   784.03   
HistGradientBoostingRegressor                0.58       0.58   785.39   
GradientBoostingRegressor                    0.55       0.56   808.11   
BaggingRe

In [79]:
from sklearn.linear_model import PoissonRegressor,LinearRegression,Ridge,Lasso
from sklearn.tree import DecisionTreeRegressor,ExtraTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor,RandomForestRegressor,HistGradientBoostingRegressor 
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
import xgboost as xgb
reg = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor",GradientBoostingRegressor()),
])
reg.fit(x_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num_features',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['chieuDai', 'chieuNgang',
                                                   'dienTich', 'Phongngu',
                                                   'SoTang', 'PhongTam']),
                                                 ('ordinal_features',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='unknown',
                                                                                 strategy='constant')),
                                                                  ('enco...
                                                                                               'cấp'],
                                                                                              ['phường '
                                                                                               'tân '
                                                                                               'tạo '
                                                                                               'a',
                                                                                               'phường '
                                                                                               'bình '
                                                                                               'hưng '
                                                                                               'hoà '
                                                                                               'b',
                                                                                               'phường '
                                                                                               'tân '
                                                                                               'tạo',
                                                                                               'phường '
                                                                                               'bình '
                                                                                               'hưng '
                                                                                               'hòa',
                                                                                               'phường '
                                                                                               'bình '
                                                                                               'trị '
                                                                                               'đông '
                                                                                               'a',
                                                                                               'phường '
                                                                                               'bình '
                                                                                               'trị '
                                                                                               'đông',
                                                                                               'phường '
                                                                                               'an '
                   

In [20]:
# parameters = {
#     "regressor__n_estimators": [50, 100, 200, 500],
#     "regressor__criterion": ["squared_error", "absolute_error", "poisson"],
#     "regressor__max_depth": [None, 5, 10, 20],
#     "regressor__max_features": ["sqrt", "log2"],
#     "preprocessor__num_features__imputer__strategy": ["mean", "median"],
# }
# model = GridSearchCV(reg, param_grid=parameters, scoring="r2", cv=6, verbose=2, n_jobs=-1)
# model.fit(x_train, y_train)
# print(model.best_score_)
# print(model.best_params_)

In [80]:
y_predict = reg.predict(x_test)

In [81]:
accuracy_list = [(i, j, abs(i - j) / i * 100 if i != 0 else 0) for i, j in zip(y_test, y_predict)]

accuracy_list_sorted = sorted(accuracy_list, key=lambda x: x[2])

for actual, predict, percentage_error in accuracy_list_sorted:
    print("Actual: {} - Predict: {} - Percentage Error: {:.2f}%".format(actual, predict, percentage_error))



Actual: 3348.333333333333 - Predict: 3348.190184943255 - Percentage Error: 0.00%
Actual: 2430.4166666666665 - Predict: 2429.7427999235265 - Percentage Error: 0.03%
Actual: 3431.25 - Predict: 3432.583364677065 - Percentage Error: 0.04%
Actual: 2864.583333333333 - Predict: 2866.6894233756566 - Percentage Error: 0.07%
Actual: 3857.9166666666665 - Predict: 3860.929792982834 - Percentage Error: 0.08%
Actual: 4322.916666666667 - Predict: 4327.301833811609 - Percentage Error: 0.10%
Actual: 3906.25 - Predict: 3912.206280347226 - Percentage Error: 0.15%
Actual: 6127.5 - Predict: 6117.746440110923 - Percentage Error: 0.16%
Actual: 3888.75 - Predict: 3895.5866117918727 - Percentage Error: 0.18%
Actual: 3472.083333333333 - Predict: 3478.346161982986 - Percentage Error: 0.18%
Actual: 3472.083333333333 - Predict: 3478.346161982986 - Percentage Error: 0.18%
Actual: 4166.666666666667 - Predict: 4157.8383737129725 - Percentage Error: 0.21%
Actual: 3445.4166666666665 - Predict: 3453.094875711192 - Perce

In [82]:
# Khởi tạo các khoảng lỗi
error_ranges = [(0, 20), (20, 50), (50, 70),(70,100),(100, float('inf'))]

# Khởi tạo danh sách để lưu kết quả đếm số lượng trong mỗi khoảng
range_counts = {f"{low}-{high}": 0 for low, high in error_ranges}

# Đếm số lượng dự đoán thuộc vào từng khoảng lỗi
for _, _, percentage_error in accuracy_list_sorted:
    for low, high in error_ranges:
        if low <= percentage_error < high:
            range_counts[f"{low}-{high}"] += 1
            break

# Tổng số lượng dự đoán
total_predictions = len(accuracy_list_sorted)

# Tính và in phần trăm cho mỗi khoảng lỗi
print("Percentage Error Ranges:")
for range_str, count in range_counts.items():
    percentage = (count / total_predictions) * 100
    print(f"Error Range {range_str}%: {count} predictions ({percentage:.2f}%)")


Percentage Error Ranges:
Error Range 0-20%: 716 predictions (72.69%)
Error Range 20-50%: 233 predictions (23.65%)
Error Range 50-70%: 24 predictions (2.44%)
Error Range 70-100%: 8 predictions (0.81%)
Error Range 100-inf%: 4 predictions (0.41%)


In [83]:
import math
print("MAE {}".format(mean_absolute_error(y_test, y_predict)))
print("RMSE {}".format(math.sqrt(mean_squared_error(y_test, y_predict))))
print("R2 {}".format(r2_score (y_test, y_predict)))

MAE 570.6573758600729
RMSE 808.2125411747594
R2 0.5556643912242692
